### Import Modules

<br>

In [1]:
import requests
import json
import time
import itertools
import wget
import os
import pickle
import numpy as np

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy

sns.set_style('white')
import tensorflow as tf
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.callbacks import LearningRateScheduler
from keras import callbacks
from keras import regularizers as reg
from keras.optimizers import SGD, Adam
from keras.models import Sequential , Model , load_model
from keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras.initializers as init
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.




<br>
### Open the Preprocessed Poster Data

<br>
Image data is stored in a numpy format and existing on a pickle dump file

<br>

In [2]:
x_test_dict = pickle.load(open('test_num.pik' , 'rb'))



<br>
### Extract data

<br>

In [3]:
# Extract the input arrays from the data container
x_test_raw = x_test_dict['images']

# transform into numpy array
x_test  = np.array(x_test_raw)




<br>
### Prepare Data for input

<br>
 - Determine the data axis order according tothe keras configuration
 - Define the input and its attributes
 - Center input data and ensure its type as float 
 - Pre-process labels
 
 <br>
 
** 1 - Arrange Data for Input:**

<br>

In [4]:
# Define the image number of rows
img_rows = x_test.shape[1]

# Dfine the image number of columns
img_cols = x_test.shape[2]

# Re-arrange if channels first configuration
if K.image_data_format() == 'channels_first':
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    
    # Define Input Shape
    input_shape = (3, img_rows, img_cols)

# Re-arrange if channels last configuration
else:
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    
    # Define Input shape
    input_shape = (img_rows, img_cols, 3)


<br>
** 2 - Pre-process Data: **

<br>

In [5]:
# Transform to float
x_test  = x_test.astype('float32')

# Center the data
x_test  /= 255.0

# Display test data attributes
print 'x_test shape:', x_test.shape
print  x_test.shape[0], 'test samples'

x_test shape: (1001, 128, 85, 3)
1001 test samples



<br>
** 3 - Output Data Pre-processing : **

<br>

In [6]:
# Extract Labels from File
y_raw = pd.read_csv('Genres_labels_All_test_cleaned2.csv')

# Split into train and test
y_test  = y_raw.iloc[:, 1:-1].values

# Define the number of classes
num_classes = y_test.shape[1]

print 'number of classes:  ' , num_classes

number of classes:   14



<br>
### Model Construction


<br>
** 1 - Import model: **

<br>

In [7]:
model = load_model('cnn1_c.h5')


<br>
** 2 - Perform Evaluation: **

<br>

In [8]:
loss , overall_acc = model.evaluate(x_test , y_test , batch_size = 16 , verbose = 1)

 992/1001 [============================>.] - ETA: 0s 

In [9]:
overall_acc

0.84080218012397223


<br>
** 3 - Perform prediction:**

<br>

In [10]:
y_predict = model.predict(x_test , batch_size = 16 , verbose = 1)

 976/1001 [============================>.] - ETA: 0s

In [11]:
# Calculate absolute error
abs_err = np.abs(y_test - y_predict)

# Calculate pprediction proximity measure (1 - absolute error) for visualization
prox = 1.0 - abs_err


<br>
### Export Results to File:

<br>
** 1 - Export prediction : **

<br>

In [12]:
# Construct data frame
proxdf = pd.DataFrame(prox , columns = y_raw.columns[1:-1]).iloc[:100 , :]

proxdf.head()

,Action,Adventure,Animation,Comedy,Crime,Drama,Family,History,Horror,Fantasy_Mystery,Romance,Science Fiction,Thriller,Other
0,0.831160,0.094347,0.956770,0.715986,0.892181,0.499974,0.934107,0.046265,0.867278,0.885098,0.862054,0.905424,0.803869,0.863633
1,0.829370,0.904267,0.955785,0.711545,0.107200,0.504288,0.931899,0.953814,0.133104,0.883800,0.861939,0.904914,0.193473,0.864418
2,0.832443,0.905821,0.956843,0.715643,0.892685,0.499909,0.934296,0.954020,0.868265,0.885632,0.861298,0.906162,0.804648,0.863734
3,0.832744,0.905947,0.956887,0.715466,0.892853,0.499467,0.934371,0.954113,0.868706,0.885588,0.861174,0.906342,0.804950,0.863988
4,0.172854,0.095765,0.044182,0.712929,0.891712,0.504031,0.931892,0.953221,0.865539,0.883214,0.863403,0.903849,0.804692,0.864632


In [13]:

# Write to File
proxdf.to_csv('cnn1c_prox.csv')


<br>
** 2 - Export proximity **

<br>

In [14]:
# Construct data frame
y_preddf = pd.DataFrame(y_predict , columns = y_raw.columns[1:-1])

# Export File
y_preddf.to_csv('cnn1cpred.csv')